In [1]:
import json
from copy import deepcopy

In [3]:
with open("../data/text/240_sampled_transcript.json", "r") as f:
    whisper = json.load(f)

In [4]:
with open("../data/text/240_sampled.json", "r") as f:
    diarization = json.load(f)

In [ ]:
texts = []
for x in whisper:
    new_start = x["start"]
    new_end = x["end"]
    new_text = x["text"]
    if text[-1] == "," and new_text[0].isupper():
        text += new_text
        end = new_end
        continue
    texts.append(
        {
            "start": start,
            "end": end,
            "text": text,
        }
    )
    start = new_start
    end = new_end
    text = new_text

In [11]:
diary = deepcopy(diarization)

transcript = []
speaker = None
for x in whisper:
    new_start = x["start"]
    new_end = x["end"]
    new_text = x["text"]
    new_speaker = None
    new_speaker_dict = {}
    for i, y in enumerate(diary):
        if  y["stop"] < new_start:
            diary.pop(i)
        elif new_start <= y["stop"] and new_end >= y["start"]:
            speaker_duration = max(y["start"],new_start) - min(y["stop"], new_end)
            new_speaker = y["speaker"]
            new_speaker_dict[new_speaker] = new_speaker_dict.get(new_speaker, 0) + speaker_duration
        elif new_end < y["start"]:
            break
    new_speaker = max(new_speaker_dict, key=new_speaker_dict.get) if len(new_speaker_dict) > 0 else "Unknown"
    
    # first speaker
    if speaker is None:
        speaker_start = new_start
        speaker_end = new_end
        speaker = new_speaker
        speaker_text = new_text
    
    # if old speaker is same as current speaker
    if new_speaker == speaker:
        speaker_text += " " + new_text
        speaker_end = new_end
    # else: speaker changed, add old speaker to transcript
    else:
        transcript.append({"start": speaker_start, "end": speaker_end, "speaker": speaker, "text": speaker_text})
        speaker_start = new_start
        speaker_end = new_end
        speaker = new_speaker
        speaker_text = new_text

In [ ]:
speaker_dict = {
    "SPEAKER_00": "Sorter",
    "SPEAKER_01": "Blacksheep",
    "SPEAKER_02": "Gergő",
    "SPEAKER_03": "FreddyD",
}